In [1]:
import csv
from math import sin, cos, sqrt, atan2, radians, asin, pi
import random
import time


# this class node to create nodes using their names and their connections
class Node:
    def __init__(self,name):
        self.name = name
        self.node_neighbors = []
        
    def connection(self,node,weight):
        self.node_neighbors.append((node.name,weight))


# this class edge is used to create edges 
class Edge:
    def __init__(self,frm,to,weight=0,directed=False):
        self.frm = frm
        self.to = to
        self.weight = weight
        self.directed = directed


# this class graph is used to construct our graph using edges from our csv file by creating nodes and connections
class Graph:
    def __init__(self,edges):
        self.nodes = {}
        self.edges ={}
        for frm, to , weight, directed in edges:
            node_frm = Node(frm)
            node_to = Node(to)
            edge = Edge(frm,to,weight,directed)
            key = (frm,to)
            self.edges[key] = edge
            if node_frm.name not in self.nodes:
                self.nodes[node_frm.name] = node_frm
                
            if node_to.name not in self.nodes:
                self.nodes[node_to.name] = node_to
           
            self.nodes[node_frm.name].connection(node_to,weight)
            if directed =="FALSE":
                self.nodes[node_to.name].connection(node_frm,weight)


# this method is used to randomly add nodes within n*(20) in to the graph by adding 6 random connections every time node is added            
    def add_node(self,node_list):
        for node in node_list:
            if node not in self.nodes:
                newNode = Node(node)
                self.nodes[str(newNode.name)] = newNode
                for _ in range(6):
                    temp = random.randint(0,19)
                    weight = random.randint(60,200)
                    randomNode = list(self.nodes.keys())[temp]
                    newNode.connection(self.nodes[randomNode],weight)
                    self.nodes[randomNode].connection(newNode,weight)


# this method is used to represent our graph as adjacency list graph representation
    def __repr__ (self):
        result = " "
        for node in self.nodes:
            result += "{} : {}\n".format(node,self.nodes[node].node_neighbors)
        return result


#  here we implement depth first search algorithm
def depth_first_algo(graph, initial_node,target_node):
    stack = [(initial_node,0)]
    visited = set()
    passed_nodes = 0
    while (stack):
        current_node, current_weight = stack.pop()
        if current_node not in visited:
            visited.add(current_node)
            passed_nodes +=1
            if current_node == target_node:
                return passed_nodes-1
            for node, weight in graph.nodes[current_node].node_neighbors:
                if node not in visited:
                    newWeight = int(weight) + int(current_weight)
                    stack.append((node,newWeight))

    raise ValueError("Target city node not in city node list!")                


# here we implement breath first search algorithm
def breadth_first_algo(graph,initial_node,target_node):
    queue = [(initial_node,0)]
    visited = set()
    passed_nodes = 0
    while (queue):
        current_node, current_weight = queue.pop(0)
        if current_node not in visited:
            passed_nodes +=1
            visited.add(current_node)
            if current_node == target_node:
                return passed_nodes-1
            for node,weight in graph.nodes[current_node].node_neighbors:
                if node not in visited:
                    newWeight = int(weight) + int(current_weight)
                    queue.append((node,newWeight))

    raise ValueError("Target city node not in city node list!")


# here we implement Dijkstra's shortest path with using priority queue           
def Dijkstra_shortest_path(graph,initial_node,target_node):

    distance = [float('inf')]* len(graph.nodes.keys())
    index_dict = {}
    
    for index in range(len(distance)):
        index_dict[list(graph.nodes.keys())[index]] = index

    queue = []
    queue.append((0,initial_node))
    index = index_dict[initial_node]
    distance[index] = 0
    passed_nodes = 0
    while(queue):
        queue.sort(reverse=False)
        current_weight, current_node = queue.pop(0)
        passed_nodes +=1
   
        if current_node == target_node:
                    return passed_nodes-1

        for value, weight in graph.nodes[current_node].node_neighbors:
            index = index_dict[value]
            newWeight = current_weight + int(weight)
            if newWeight <= distance[index]:
                distance[index] = newWeight
                for i,j in queue:
                    if value == j and i > newWeight:
                        index = queue.index((i,j))
                        queue[index] = (newWeight,j)

                if (newWeight,value) not in queue:
                    queue.append((newWeight,value))

                 

# this fuction is used to print the shortest distance between any two initial node and target node
def short_path_print(distance, index_dict,target_node):
    result =""
    initial = list(index_dict.keys())[0]
    for value in index_dict:
        if value == target_node:
            result += "{} to {} is {}\n".format(initial, value,distance[index_dict[value]])
            break
    return result

# this function is used to calculate the heuristic value for each nodes using latitude and longitude
def heuristic_value_calculate(lat_long,graph,target_node):
    heuristic_distance = {}
    for _ in range(len(list(lat_long.keys()))):
        node = list(lat_long.keys())[_]
        if node == target_node:
            heuristic_distance[target_node] = float('inf')
        value = distance(lat_long[node][0],lat_long[node][1],lat_long[target_node][0],lat_long[target_node][1])
        heuristic_distance[node] = value

    return heuristic_distance

# here we implement A * shortest path algorithm using priority queue
def A_star_path_finding(graph,heuristic_distance, initial_node,target_node):
    
    distance = [float('inf')]* len(graph.nodes.keys())
    index_dict = {}
    
    for index in range(len(distance)):
        index_dict[list(graph.nodes.keys())[index]] = index
        
    queue = [(0+heuristic_distance[initial_node], initial_node)]
    index = index_dict[initial_node]
    distance[index] = 0
    passed_nodes = 0
    while(queue):
        queue.sort(reverse=False)
        current_weight_heuristic_value,current_node = queue.pop(0)
        passed_nodes+=1

        if current_node == target_node:
                return passed_nodes-1

        for value, weight in graph.nodes[current_node].node_neighbors:
            index = index_dict[value]
            newWeight = current_weight_heuristic_value-heuristic_distance[current_node] + int(weight)
            
            if newWeight <= distance[index]:
                distance[index] = newWeight 
                for i,j in queue:
                    if value == j and i > newWeight:
                        index = queue.index((i,j))
                        queue.pop(index)
                        newWeight += heuristic_distance[value]
                        queue.append((newWeight,j))
                        
                if (newWeight,value) not in queue:
                    newWeight += heuristic_distance[value]
                    queue.append((newWeight,value))
    
    raise ValueError("Target city node not in city node list!")


# this function is used to calculate the distance between any to nodes using latitude and longitude points
def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    
    return 12742 * asin(sqrt(a))


# here goes the implementation for reading csv file of edge list's
file = open('City_Edges_list.csv')
csvreader = csv.reader(file)
header = []
header = next(csvreader)
edges = []
# we add all the edges list into edges dictionary
for edge in csvreader:
        edges.append(tuple(edge))


# here goes the implementation for reading csv file of each city/node latitude and longitude       
file = open('cities_latitude_longitudes.csv')
csvreader = csv.reader(file)
header = []
header = next(csvreader)
lat_long_list = {}
# we add all the latitude and longitude as a tuple in to lat_long_list
for city,latitude,longitude in csvreader:
    lat_long_list[city] = (float(latitude),float(longitude))


# we use node_list to randomly add nodes in to the graph where num stands for how many * of the original size number of random nodes to be inserted
# if num == 1 the the total number of nodes will be 40 which is 2* the original size, which is 20 
# this function is used to add random nodes to our graph
def add_random_nodes(num):
    size = len(list(graph.nodes.keys()))
    node_list = [ str(_) for _ in range(num*size)]
    # we use .add_node to create and add nodes
    graph.add_node(node_list)

    
graph = Graph(edges)
# adding n*original size nodes
add_random_nodes(0)


# this function adds heuristic values to the newly added nodes using random latitude and longitudes with in range of 21.0 - 28.0 and 43.0 - 48.0 respectively
def add_heuristic_values(lat_long_list):
    for node in list(graph.nodes):
        if node not in lat_long_list:
            for _ in range(2):
                lat = random.uniform(43.0, 48.0)
                long = random.uniform(21.0, 28.0)
                lat_long_list[node] = (lat,long)
    return lat_long_list


# to updated lat_long_list by including latitude and longitude values for newly added nodes
lat_long_list = add_heuristic_values(lat_long_list)


# this function calculate the average distance and average time taken for each and every node
def average_distance_time_value(search_algorithm,graph,total_nodes):
    total_distance = 0
    total_time_taken = 0

    for initial_node in list(graph.nodes.keys()):
        for target_node in list(graph.nodes.keys()):
            start = time.time()
            total_distance+=search_algorithm(graph,initial_node,target_node)
            end = time.time()
            total_time_taken += (end -start)

    average_time_taken = total_time_taken/((total_nodes*total_nodes)-total_nodes)
    average_distance = total_distance/((total_nodes*total_nodes)-total_nodes)
    return (average_distance, average_time_taken)


# this function calculate the average distance and average time taken for each and every node for A* search algorithm
def average_distance_time_value_A_star(search_algorithm,graph,total_nodes):
    total_distance = 0
    total_time_taken = 0

    for initial_node in list(graph.nodes.keys()):
        for target_node in list(graph.nodes.keys()):
            heuristic_distance = heuristic_value_calculate(lat_long_list,graph,target_node)
            start = time.time()
            total_distance+=search_algorithm(graph,heuristic_distance,initial_node,target_node)
            end = time.time()
            total_time_taken += (end -start)

    average_time_taken = total_time_taken/((total_nodes*total_nodes)-total_nodes)
    average_distance = total_distance/((total_nodes*total_nodes)-total_nodes)
    return (average_distance, average_time_taken)


# getting the total number of nodes
total_nodes = len(graph.nodes.keys())


# passing DFS, BFS and Dijkstra's search algorithms for benchmarking their results
distance_ = 0
time_ = 0
for i in range(100):
    x, y = average_distance_time_value(depth_first_algo,graph,total_nodes)
    distance_ += x
    time_ +=y
print(distance_/100, time_/100)


# passing DFS, BFS and Dijkstra's search algorithms for benchmarking their results
distance_ = 0
time_ = 0
for i in range(100):
    x, y = average_distance_time_value(breadth_first_algo,graph,total_nodes)
    distance_ += x
    time_ +=y
print(distance_/100, time_/100)


# passing DFS, BFS and Dijkstra's search algorithms for benchmarking their results
distance_ = 0
time_ = 0
for i in range(100):
    x, y = average_distance_time_value(Dijkstra_shortest_path,graph,total_nodes)
    distance_ += x
    time_ +=y
print(distance_/100, time_/100)



# passing A* function to benchmark the results we get
distance_ = 0
time_ = 0
for i in range(10):
    x, y = average_distance_time_value_A_star(A_star_path_finding,graph,total_nodes)
    distance_ += x
    time_ +=y
print(distance_/100, time_/100)



10.0 1.3721685660512823e-05
10.0 1.716434955596924e-05
10.0 4.0913594396490794e-05
0.5386842105263157 3.0297731098375817e-06


In [221]:
num1 = 10122.231578947369
num2 = 10038.228947368421
num3 = 10094.807894736841
num4 = 10099.160526315789
num5 = 10235.88947368421
# num6 = 409.64473684210526
# num7 = 409.64473684210526
# num8 = 409.64473684210526
# num9 = 409.64473684210526
# num10 = 409.64473684210526
print((num1+num2+num3+num4+num5)/5)

10118.063684210527


In [222]:
tem1 = 0.03863005638122559
tem2 = 0.039473844829358555
tem3 = 0.04130198390860307
tem4 = 0.038551680037849825
tem5 = 0.04027605244987889
# tem6 = 4.206205669202303e-05
# tem7 = 3.9392396023398955e-05
# tem8 = 3.687143325805664e-05
# tem9 = 1.573123429950915e-05
# tem10 = 3.169147591841848e-05
print((tem1 +tem2 +tem3 +tem4 +tem5)/5)
# print(2.7200799239309213e-05-4.158296083149158e-05)

0.03964672352138318
